In [10]:
from ml_fingerprint import ml_fingerprint, example_models, remote
import sqlite3
from Crypto.PublicKey import RSA

ml_fingerprint.decorate_base_estimator()

In [11]:
from datetime import datetime
d = datetime.now()

In [12]:
model, score = example_models.rain_classifier()

scores = {'Accuracy': score}
description = '''Linear classifier that predicts if it will rain tomorrow.
    The dataset is taken from Kaggle and can be found here:
    https://www.kaggle.com/jsphyg/weather-dataset-rattle-package
    It consists of daily weather data in australian cities, and whether
    it rained the next day or not.'''

In [14]:
database = 'flask/ml_fingerprint_database.db'
conn = sqlite3.connect(database)
conn.row_factory = sqlite3.Row
c = conn.cursor()
db_key = c.execute('select * from key order by id desc limit 1').fetchone()
public_key = RSA.import_key(db_key['publickey'])
private_key = RSA.import_key(db_key['privatekey'])

In [15]:
model.sign(private_key)

Signing model...


b"A\xb4\xd0\x97;\x84\xfe\x06`\xcf\x0fqpk\xbc\x88N\xa5\xf6\xe7\xeaK\x98C\xf7\x8d\xee\x18T\x9c<\xe91C\x12\xff\xe6\xc5\xe51PB\x1bU\x8f9\xee\x82\x17\xa1\x0c\xbb\x18G\xfe\xa1\xea\xebC\xf9BO\x8e\x9e\xe3u_r\x89\x01\x88>D{\xa6\xf5\xb6\xa9\x9dz\x9c\xaam\x1e\xa0\xa5vVe#\xc3\x99\x1c6\x19.\xf7\x13\xfe\x12\x1aXr\xc9xD\x08\x97`\x1d\xdf\xa1\xa3\x9a\x9e\xc0\xe1\xa9\xb0\x98^*\xbf\xfdY\x8c\x82\xc2fc\xf1A\x8b[o\x0bHF\x9a\x05\x85\x9c\xc5\x1d\x8f\x97|\xec\x94%=H\xab\x15\xbfe\x1bl\xa4\xcc\x11PK[!L\x974[c\x81\xc9\xab\x81\xda\xe5#\x95\xab\xfe\xe2\xf8*\x92\x9bX\x11O\x00\x06\x92q\xd45/Y\xa1;\xc5\x14\x19H\x99b\x03\xd6 \xb9\xc4\x85\xc9qN]\xcfX@`t\x10J'\xb7p\xd9h\xaf\xc2'\x9bl\xdbH']i\x8d\xdf\xde\x07\x144\xd6T\xf3\xb7\xcd\x95\x10\x1a\x85@m\xd3\x83\xa9"

In [16]:
server = remote.RemoteServer('http://localhost:5000')

In [17]:
server.insert_model(model, 'australia_rain_predict', True, 'Classification', scores, '1.0.0', {}, datetime.now(), description, "Jaime Solsona")

In [21]:
model = example_models.vanderplas_regression()

description = '''Example linear regressor taken from "Python Data Science Handbook", by Jake VanderPlas.
    The training points follow the function y = -2x + z, with an added noise.
    The regressor then tries to predict the value of y based on x and z values.'''

model.sign(private_key)

Signing model...


b'\x7f\xeb\x1b\x14n[\x12b\x82\x02W\x07SBQ\x0b\xf7\x9b~\xeeq\\\x9f+\x98`\x006\xa5_\xf3\x07\x81z\xc2Mn0K/e\xdc@@\x94eod\\\xcf~\x0ey,\r\x05\xae>\xaci\xe9\xe3\xf3\xdb\xe8\x8fn\xf7|\x9ec\xa2O\x9e73\xf4 \xe6\xe5\xab \xeb\x19^\xc0\x93\xda\x9f-B\xb9d<\xefu\x1d\xb6;[\xcc\xb2}\x1b\xb28\xfa\xa0[\x90N)\x9c\xdb\xe5\xdfv;\xeeq\xc1\xbd\xacrU\xc7LiZ\xbd\xe4\xb0\x95\x9b$\xe3\xd5\x949\xb0\x9a\x88\xeap\xe8\xf3\xb5\x87\xe6U\xeaOVO\xc1\xd5]Nr\xaaKb,\x95%tu\x81\x0cC\xb6\x13\xbf\xf2rG\t\xb0\xad\xb7P\xab\x87\x81c\xfc\x86\xf4\x00jy\xe7\xb4\x8a"\xb9\x9c\xb9\xe6)\xa89>W\x17\xac\xb4>\x05P\x97x\xad\xebi\xb1\xc1Z\x83`t\xd3\xff\x02\xd1\xfb0N\xfc\xa5\x97\xbdm\x7fqXh.\'Ih\xbc\xb7:\xce"\x9a\xcd}\x0c\xbbY\xbd\x91\xc5n'

In [22]:
import numpy as np
rng = np.random.RandomState(1)
X = rng.randn(100, 2)
y = -2*X[:,0] + X[:,1]
y_pred = model.predict(X)



In [23]:
from sklearn import metrics
mse = metrics.mean_squared_error(y, y_pred)
r2 = metrics.r2_score(y, y_pred)
scores = {}
scores['MSE'] = mse
scores['R2'] = r2

In [24]:
server.insert_model(model, 'vanderplas_regression_example', True, 'Regression', scores, '1.0.0', {}, datetime.now(), description, "Jaime Solsona")

In [25]:
model = server.get_model('australia_rain_predict', public_key)

Validating model...
The signature is valid


In [26]:
model.__dict__

{'penalty': 'l2',
 'dual': False,
 'tol': 0.0001,
 'C': 1.0,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'class_weight': None,
 'random_state': 0,
 'solver': 'liblinear',
 'max_iter': 100,
 'multi_class': 'auto',
 'verbose': 0,
 'warm_start': False,
 'n_jobs': None,
 'l1_ratio': None,
 'n_features_in_': 59,
 'classes_': array(['No', 'Yes'], dtype=object),
 'coef_': array([[ 7.34061869e-01,  4.83591525e+00, -1.29798540e+00,
          6.39496454e+00,  7.18226328e+00, -1.13994767e+01,
          1.74735264e+00,  6.11210126e-02,  4.55624380e-02,
         -4.79409494e-02,  6.32295809e-01,  1.02049615e-02,
          2.54685474e-01, -1.57357454e-01,  7.23437761e-02,
         -6.31998236e-01, -1.14041237e-01,  3.40381419e-01,
         -1.57619687e-01,  8.65198796e-02,  1.37248169e-01,
         -6.78893544e-02,  1.26669710e-01, -3.88894835e-01,
         -4.73751079e-01, -4.45854331e-01, -6.93374097e-01,
         -3.38270657e-01,  8.02738789e-02, -3.74998434e-01,
          3.59617136e-02,  

In [27]:
model = server.get_model('vanderplas_regression_example', public_key)

import numpy as np
rng = np.random.RandomState(1)
X = rng.randn(100, 2)
y = -2*X[:,0] + X[:,1]
y_pred = model.predict(X)

from sklearn import metrics
r2 = metrics.r2_score(y, y_pred)

print(r2)


Validating model...
The signature is valid
0.9999320997661125


In [ ]:
model = server.delete_model('australia_rain_predict')
model = server.delete_model('vanderplas_regression_example')